# Minimal Blockchain in Python

In this notebook, I will demonstrate how you can build a minimal private blockchain with essential operations such as:

a) creating a blockchain, b) verifying a chain, c) forking, and d) comparing chains.

In [30]:
import copy  # fork a chain
import datetime  # get real time for timestamps
import hashlib  # hash
from enum import IntEnum
from datetime import timedelta  # used for obtaining time deltas between timestamps of blocks

## 1. Define classes

In [31]:
# forking enums are used by the
# fork method of the MinimalChain
# class so the head input parm
# would be a consistent integer
class ForkingEnums(IntEnum):
    ALL = -1
    LATEST = -2
    WHOLE = -3

In [32]:
class MinimalBlock:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hashing()

    def __repr__(self):
        value = f'index: {self.index}, timestamp: {self.timestamp}, data: {self.data}, prev_hash: {self.previous_hash}, this_hash: {self.hash}'
        return value

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def hashing(self):
        key = hashlib.sha256()
        key.update(str(self.index).encode('utf-8'))
        key.update(str(self.timestamp).encode('utf-8'))
        key.update(str(self.data).encode('utf-8'))
        key.update(str(self.previous_hash).encode('utf-8'))
        return key.hexdigest()

    def verify(self):  # check data types of all info in a block
        instances = [self.index, self.timestamp, self.previous_hash, self.hash]
        types = [int, datetime.datetime, str, str]
        if sum(map(lambda inst_, type_: isinstance(inst_, type_), instances, types)) == len(instances):
            return True
        else:
            return False

In [33]:
class MinimalChain:
    def __init__(self):  # initialize when creating a chain
        self.blocks = [MinimalChain.get_genesis_block()]

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    @staticmethod
    def get_genesis_block():
        return MinimalBlock(0,
                            datetime.datetime.utcnow(),
                            'Genesis',
                            'arbitrary')

    def add_block(self, data):
        minimal_block = MinimalBlock(len(self.blocks),
                                     datetime.datetime.utcnow(),
                                     data,
                                     self.blocks[len(self.blocks) - 1].hash)
        self.blocks.append(minimal_block)
        print(minimal_block)

    def get_chain_size(self):  # exclude genesis block
        return len(self.blocks) - 1

    def verify(self, verbose=True):
        flag = True
        for ndx in range(1, len(self.blocks)):
            if not self.blocks[ndx].verify():  # assume Genesis block integrity
                flag = False
                if verbose:
                    print(f'Wrong data type(s) at block {ndx}.')
            if self.blocks[ndx].index != ndx:
                flag = False
                if verbose:
                    print(f'Wrong block index at block {ndx}.')
            if self.blocks[ndx - 1].hash != self.blocks[ndx].previous_hash:
                flag = False
                if verbose:
                    print(f'Wrong previous hash at block {ndx}.')
            if self.blocks[ndx].hash != self.blocks[ndx].hashing():
                flag = False
                if verbose:
                    print(f'Wrong hash at block {ndx}.')
            ts_curr = self.blocks[ndx].timestamp
            ts_prev = self.blocks[ndx - 1].timestamp
            ts_diff = (ts_curr - ts_prev) / timedelta(microseconds=1)
            if ts_diff < 0:
                flag = False
                if verbose:
                    print(f'Backdating at block {ndx}.')
        return flag

    def fork(self, head=ForkingEnums.LATEST.value):
        if head in [ForkingEnums.LATEST.value, ForkingEnums.WHOLE.value, ForkingEnums.ALL.value]:
            return copy.deepcopy(self)  # deepcopy since they are mutable
        else:
            cpy = copy.deepcopy(self)
            cpy.blocks = cpy.blocks[0:head + 1]
            return cpy

    def get_root(self, chain_2):
        min_chain_size = min(self.get_chain_size(), chain_2.get_chain_size())
        for i in range(1, min_chain_size + 1):
            if self.blocks[i] != chain_2.blocks[i]:
                return self.fork(i - 1)
        return self.fork(min_chain_size)

## 2. Testing

In [34]:
# Testing

print('')
print('Testing...')
print('')


Testing...



In [35]:
nbr_of_blocks = 10

c = MinimalChain()  # Start a chain
for i in range(1, nbr_of_blocks + 1):
    c.add_block(f'Block {i} of the chain.')

index: 1, timestamp: 2019-09-27 19:05:41.473334, data: Block 1 of the chain., prev_hash: f1de49a0ea7474d46c423ae3674bea23abc98a94cdb18b84efe8958acf929593, this_hash: aaeca83893eb6003fdb6e347381b669f217d85f4f6fb240cd9670c19040eeb76
index: 2, timestamp: 2019-09-27 19:05:41.474334, data: Block 2 of the chain., prev_hash: aaeca83893eb6003fdb6e347381b669f217d85f4f6fb240cd9670c19040eeb76, this_hash: 3fdfd797da2bd34b3386168dd1799dec0b6652887b626d4ede82d804305409b8
index: 3, timestamp: 2019-09-27 19:05:41.474334, data: Block 3 of the chain., prev_hash: 3fdfd797da2bd34b3386168dd1799dec0b6652887b626d4ede82d804305409b8, this_hash: 4f28908f7e331b6e9683adce4031c23acc5bc7928d020973b072fd5b47725eb6
index: 4, timestamp: 2019-09-27 19:05:41.474334, data: Block 4 of the chain., prev_hash: 4f28908f7e331b6e9683adce4031c23acc5bc7928d020973b072fd5b47725eb6, this_hash: 961fb2a0868bd788f0202f6457ab751a42982938ba6e8e8f6d1ee1b7e5c97db3
index: 5, timestamp: 2019-09-27 19:05:41.474334, data: Block 5 of the chain.

In [36]:
print(f'c.blocks[3].timestamp: {c.blocks[3].timestamp}')
print(f'c.blocks[7].data: {c.blocks[7].data}')
print(f'c.blocks[9].hash: {c.blocks[9].hash}')

c.blocks[3].timestamp: 2019-09-27 19:05:41.474334
c.blocks[7].data: Block 7 of the chain.
c.blocks[9].hash: 50b3d614b3f5fbf4d62b1d2d9e3d5c3304e0d711c3a6ef5e075e23635dbe833c


In [37]:
print(f'c.get_chain_size(): {c.get_chain_size()}')
print(f'Verifying {c.get_chain_size()} blocks...')
print(f'c.verify(): {c.verify()}')

c.get_chain_size(): 10
Verifying 10 blocks...
c.verify(): True


In [38]:
c_forked = c.fork(ForkingEnums.LATEST.value)
print(f'c == c_forked: {c == c_forked}')

c == c_forked: True


In [39]:
c_forked.add_block('New block for forked chain!')
print(c.get_chain_size(), c_forked.get_chain_size())

index: 11, timestamp: 2019-09-27 19:05:41.521336, data: New block for forked chain!, prev_hash: 620c0add3a9fd4353e1cec30de88d8f29655f36177b9575f933cfcc06ce15906, this_hash: b849da382a501aa77280fbb2ccd80d75183d9a28f3bf66e189420328854790a1
10 11


## 3. Conflict Testing

In [40]:
# Conflict testing

print('')
print('Conflict Testing...')
print('')


Conflict Testing...



In [41]:
c_forked = c.fork(ForkingEnums.LATEST.value)
c_forked.blocks[9].index = -9
c_forked.verify()

Wrong block index at block 9.
Wrong hash at block 9.


False

In [42]:
c_forked = c.fork(ForkingEnums.LATEST.value)
c_forked.blocks[6].timestamp = datetime.datetime(2000, 1, 1, 0, 0, 0, 0)
c_forked.verify()

Wrong hash at block 6.
Backdating at block 6.


False

In [43]:
c_forked = c.fork(ForkingEnums.LATEST.value)
c_forked.blocks[5].previous_hash = c_forked.blocks[1].hash
c_forked.verify()

Wrong previous hash at block 5.
Wrong hash at block 5.


False